In [1]:
import os
import librosa
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
import librosa.display
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LSTM, TimeDistributed, SimpleRNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Reshape, Conv1D, MaxPooling1D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [4]:
# Paths
MFCC_PATH = r"D:/FYP/Datasets/MFCC_Features"

def extract_label(filename):
    if filename.startswith("1-"):
        return "noise"
    match = re.match(r"([A-Za-z0-9]+)_", filename)
    return match.group(1) if match else None

# Load MFCC features and labels
X, y = [], []
for file in os.listdir(MFCC_PATH):
    if file.endswith(".npy"):
        file_path = os.path.join(MFCC_PATH, file)
        mfcc_features = np.load(file_path)  # Load MFCC array

        # Flatten MFCC array into a 1D feature vector
        X.append(mfcc_features.flatten())

        # Extract drone type as label from filename
        label = extract_label(file)
        if label:
            y.append(label)

# Convert to numpy arrays
X = np.array(X)
y = np.array(y)

# Encode labels numerically
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert labels to numeric values
y_categorical = to_categorical(y_encoded)   # Convert to one-hot format for classification

# Reshape MFCC input for CNN/RNN
X = np.expand_dims(X, axis=-1)  # Ensure 4D shape for CNN (samples, n_mfcc, time_frames, 1)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)
input_shape = (X.shape[1], X.shape[2], 1)  # Corrected input shape for CNN


In [3]:
# Train and evaluate models
def train_and_evaluate(model, name):
    print(f"\nTraining {name} Model...\n")
    history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"{name} Accuracy: {test_acc * 100:.2f}%")

In [4]:
def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding="same", input_shape=(X.shape[1], X.shape[2], 1)),  # Add padding to maintain size
        MaxPooling2D((2, 1)),  # Reduce only frequency dimension, keep time frames intact
        Conv2D(64, (3, 3), activation='relu', padding="same"),  
        MaxPooling2D((2, 1)),  
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(len(label_encoder.classes_), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and Evaluate Each Model
cnn_model = build_cnn()
train_and_evaluate(cnn_model, "CNN")

d:\FYP\Final Project\.venv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Training CNN Model...

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 11s 96ms/step - accuracy: 0.2737 - loss: 11.7230 - val_accuracy: 0.5249 - val_loss: 1.0717
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.5613 - loss: 1.0801 - val_accuracy: 0.6519 - val_loss: 0.7370
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 86ms/step - accuracy: 0.7292 - loss: 0.6541 - val_accuracy: 0.7735 - val_loss: 0.5195
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.8527 - loss: 0.4071 - val_accuracy: 0.8066 - val_loss: 0.4891
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - accuracy: 0.8701 - loss: 0.4040 - val_accuracy: 0.9227 - val_loss: 0.2190
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.9331 - loss: 0.2050 - val_accuracy: 0.9061 - val_loss: 0.2309
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - accuracy: 0.9383 - loss: 0.2025 - val_accuracy: 0.9503 - val_loss: 0.1921
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - accuracy: 0.9529 - loss: 0.1476 